In [1]:
import pandas as pd
import csv
import os

import linecache
import sys

import matplotlib.pyplot as plt
import math
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import plotly.graph_objs as go

#from prettytable import PrettyTable

#簡轉繁用 
#https://pypi.python.org/pypi/hanziconv/0.2.1
from hanziconv import HanziConv

import matplotlib_venn 
from matplotlib_venn import venn2, venn2_circles


import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error
from numpy import mean, sqrt, square, arange

import pickle

import random
import datetime
import operator

#https://stackoverflow.com/questions/14519177/python-exception-handling-line-number
def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print ('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))


In [46]:
def add_pin_in_column(character, mode):
    #mode 1 = sonin
    #mode 2 = muin
    if character == -1:
        return -1
    term = Totalname_list[character]
    try:
        if term not in moe_data_dict:
            term = HanziConv.toTraditional(term)
        if term in moe_data_dict:
            for hete in moe_data_dict[term]['heteronyms']:
                if 'pinyin' in hete and  moe_data_dict[term]['title']!='啐':
                    word_p = hete['pinyin']
                    if '（'  in (hete['pinyin']):
                        if '（讀音）' in hete['pinyin']:
                            word_p = hete['pinyin'].replace('（讀音）','')
                        if '（語音）' in hete['pinyin']:
                            word_p = hete['pinyin'].replace('（語音）','')   
                        if '(' in word_p:
                            print(word_p + "!!")  
                    for mu in mu_in_list:
                        if mu in word_p:
                            if mode == 'sonin':
                                return word_p[:word_p.index(mu)]
                            else:
                                return mu                                    
            #找不到字音，看是否是哪個字的異體字
            for hete in moe_data_dict[term]['heteronyms']:
                for define in hete['definitions']:
                    if '異體字' in define['def']:
                        d =  define['def']
                        alt_term = d[  d.index('「')+1 : d.index('」') ]
                        #print(alt_term,term)
                        for hete2 in moe_data_dict[alt_term]['heteronyms']:
                            if 'pinyin' in hete2 and  moe_data_dict[alt_term]['title']!='啐':
                                word_p = hete2['pinyin']
                                if '（'  in (hete2['pinyin']):
                                    if '（讀音）' in hete2['pinyin']:
                                        word_p = hete2['pinyin'].replace('（讀音）','')
                                    if '（語音）' in hete2['pinyin']:
                                        word_p = hete2['pinyin'].replace('（語音）','')   
                                    if '(' in word_p:
                                        print(word_p + "!!")
                                for mu in mu_in_list:
                                    if mu in word_p:
                                        if mode == 'sonin':
                                            return word_p[:word_p.index(mu)]
                                        else:
                                            return mu     
            #print('在字典內但沒有拼音：',term)
        else:
            #print('不在字典內：',term)
            if term in special_word_dict:
                word_p = special_word_dict[term]['pinyin']
                for mu in mu_in_list:
                    if mu in word_p:
                        if mode == 'sonin':
                            return word_p[:word_p.index(mu)]
                        else:
                            return mu   
            else:
                #print('拼音不明：',term)
                if term not in unkown_dict:
                    unkown_dict[term] = 1 
                else:
                    unkown_dict[term] += 1

#             if len(moe_df[moe_df.字詞名.apply(lambda x: x==term)])>0:
#                 print('不在字典內：',term)
    except Exception as e:
        print (e)
        PrintException() 

def add_pin_in_index_column(pin_yin, mode):
    #mode 1 = sonin
    #mode 2 = muin
    if pin_yin == None:
        return -1
    try:
        if mode == 'muin':
            if pin_yin in mu_in_list:
                return mu_in_list.index(pin_yin)
            else:
                mu_in_list.append(pin_yin)
                return mu_in_list.index(pin_yin)

        if mode == 'sonin':
            if pin_yin in son_in_list:
                return son_in_list.index(pin_yin)
            else:
                son_in_list.append(pin_yin)
                return son_in_list.index(pin_yin)

    except Exception as e:
        print (e)
        print (fileName)
        PrintException()

In [2]:
# READ TAIWAN DATA WITH FEATURES
taiwandata = pd.read_csv('./NameData/EnglishTaiwanFeatureData.csv', parse_dates = True)
resample_taiwandata = pd.read_csv('./NameData/EnglishResample_TaiwanFeatureData.csv', parse_dates = True)

In [3]:
resample_taiwandata = resample_taiwandata.rename(columns = {"E_Name": "name", "Gender_index": "GuessedGender", 
                                                            "EN_Surname": "LastName","EN_Firstname": "FirstName", 
                                                            "YearRange": "BirthYear", "Year": "message"})

In [65]:
Path = './NameData/resampled_gender.csv'
resampled_gender = pd.read_csv(Path, dtype = 'str')

In [50]:
with open('./util_files/Totalname_list', 'rb') as handle:
    Totalname_list = pickle.loads(handle.read())
    
with open('./util_files/son_in_list', 'rb') as handle:
    son_in_list = pickle.loads(handle.read())

with open('./util_files/moe_data_dict', 'rb') as handle:
    moe_data_dict = pickle.loads(handle.read())
    
with open('./util_files/mu_in_list', 'rb') as handle:
    mu_in_list = pickle.loads(handle.read())

In [66]:
resampled_gender = resampled_gender.drop('userID',1)
resampled_gender = resampled_gender.drop('FN2_radical',1)
resampled_gender = resampled_gender.drop('FN1_radical',1)
resampled_gender = resampled_gender.drop('FN2_radical_index',1)
resampled_gender = resampled_gender.drop('FN1_radical_index',1)
resampled_gender = resampled_gender.drop('天格',1)
resampled_gender = resampled_gender.drop('人格',1)
resampled_gender = resampled_gender.drop('地格',1)
resampled_gender = resampled_gender.drop('外格',1)
resampled_gender = resampled_gender.drop('總格',1)


# Separate muin

In [56]:
mu_in_character = [['ā', 'ē','ī','ō','ū'], ['ǎ','ě','ǐ','ǒ','ǔ'], ['á','é','í','ó','ú'], ['à','è','ì','ò','ù']]
in_1 = ['ā', 'ē','ī','ō','ū']
in_2 = ['ǎ','ě','ǐ','ǒ','ǔ']
in_3 = ['á','é','í','ó','ú']
in_4 = ['à','è','ì','ò','ù']
in_5 = ['a','e','i','o','u']
in_s = ['ề', 'ǘ', 'ǚ', 'ǜ']
mu_dict = {}

In [57]:
mu_dict = {}
for mc in mu_in_character :
    for index,m in enumerate(mc):
        if index==0:
            mu_dict[m]='a'
        if index==1:
            mu_dict[m]='e'
        if index==2:
            mu_dict[m]='i'
        if index==3:
            mu_dict[m]='o'
        if index==4:
            mu_dict[m]='u'
mu_dict['ề'] = 'e'
mu_dict['ǘ'] = 'u'
mu_dict['ǚ'] = 'u'
mu_dict['ǜ'] = 'u'
            
# mu_dict

In [58]:
muin_dict = {}

for i in range(5):
    muin_dict[in_1[i]] = 1
    muin_dict[in_2[i]] = 2
    muin_dict[in_3[i]] = 3
    muin_dict[in_4[i]] = 4
    #muin_dict[in_5[i]] = 5
muin_dict['ề'] = 6
muin_dict['ǘ'] = 7
muin_dict['ǚ'] = 8
muin_dict['ǜ'] = 9
# muin_dict

In [67]:
resampled_gender['FN1_muin'] = resampled_gender['FN1_muin'].fillna(-1)
resampled_gender['FN2_muin'] = resampled_gender['FN2_muin'].fillna(-1)

In [68]:
def seperate_pinjer(df,FN):
    mu_list = df[FN+'_muin'].tolist()
    new_mu_list = []
    for index,i in enumerate(mu_list):
        changed = False
        if i ==-1 or i=='-1':
            new_mu_list.append(i)
            continue
        for mu in muin_dict:
            if mu in i:
                #print(index,mu,i,muin_dict[mu])
                new_mu_list.append(i.replace(mu,mu_dict[mu]))
                changed = True
                break
        if changed == False :
            print(i)
            new_mu_list.append(i)
    df[FN+'_mu_in'] = new_mu_list
    return df

resampled_gender = seperate_pinjer(resampled_gender , 'FN1')
resampled_gender = seperate_pinjer(resampled_gender , 'FN2')


e
en
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [71]:
# Only use 'mu' without then 'in'
resampled_gender['FN1_muin'] = resampled_gender['FN1_mu_in']
resampled_gender['FN2_muin'] = resampled_gender['FN2_mu_in']
resampled_gender = resampled_gender.drop('FN1_mu_in',1)
resampled_gender = resampled_gender.drop('FN2_mu_in',1)

resampled_gender['FN1_muin_index'] = resampled_gender.FN1_muin.apply(lambda x: add_pin_in_index_column(x,'muin'))
resampled_gender['FN2_muin_index'] = resampled_gender.FN2_muin.apply(lambda x: add_pin_in_index_column(x,'muin'))

In [73]:
resampled_gender.head()

,FirstName,LastName,gender,message,name,FN1,FN2,mgender,FN1_sonin,FN1_muin,FN2_sonin,FN2_muin,FN1_sonin_index,FN2_sonin_index,FN1_muin_index,FN2_muin_index
0,民任,吳,1,1947,吳民任,335,297,1,m,in,r,en,11,26,75,76
1,昭呈,劉,1,1965,劉昭呈,462,111,1,zh,ao,ch,eng,8,21,74,25
2,恩田,莊,1,1959,莊恩田,86,202,1,NaN,en,t,ian,2,0,76,21
3,呈祥,林,1,1951,林呈祥,111,165,1,ch,eng,x,iang,21,16,25,2
4,碧山,廖,1,1944,廖碧山,1287,447,1,b,i,sh,an,7,5,164,80


# RFC Gender

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

def dataset_statistics(dataset):
    """
    Basic statistics of the dataset
    :param dataset: Pandas dataframe
    :return: None, print the basic statistics of the dataset
    """
    print (dataset.describe())
    
#http://dataaspirant.com/2017/06/26/random-forest-classifier-python-scikit-learn/
#Split train and test
def split_dataset(dataset, train_percentage, feature_headers, target_header):
    """
    Split the dataset with train_percentage
    :param dataset:
    :param train_percentage:
    :param feature_headers:
    :param target_header:
    :return: train_x, test_x, train_y, test_y
    """

    # Split dataset into train and test dataset
    train_x, test_x, train_y, test_y = train_test_split(dataset[feature_headers], dataset[target_header],
                                                        train_size = train_percentage)
    return train_x, test_x, train_y, test_y

def random_forest_classifier(features, target):
    """
    To train the random forest classifier with features and target data
    :param features:
    :param target:
    :return: trained random forest classifier
    """
    clf = RandomForestClassifier(n_estimators = 33, n_jobs = -1 , min_samples_leaf = 1)
    clf.fit(features, target)
    return clf

In [6]:
def basic_RFClassifier(data_df,x_feature,y_feature,Validation_times):

    #N fold cross-validation
    Train_accuracy_score = 0
    Test_accuracy_score = 0
    
    for i in range(Validation_times):
        train_x, test_x, train_y, test_y = split_dataset(data_df, 0.7, x_feature, y_feature)
 
        # Create random forest classifier instance
        trained_model = random_forest_classifier(train_x, train_y)
        
        predictions = trained_model.predict(test_x)
#         print(predictions)

        # Train and Test Accuracy
        if(Validation_times!=1):
            print('Run times:', i+1)
        print ("Train Accuracy :: ", accuracy_score(train_y, trained_model.predict(train_x)))
        print ("Test Accuracy  :: ", accuracy_score(test_y, predictions))
        #print ("Confusion matrix\n ", confusion_matrix(test_y, predictions))

        Train_accuracy_score += accuracy_score(train_y, trained_model.predict(train_x))
        Test_accuracy_score += accuracy_score(test_y, predictions)
        #calculate_error(predictions, test_y )
        #validate_error(predictions, test_y, test_x , data_df)
        
    if(Validation_times != 1):
        Train_accuracy_score /= Validation_times
        Test_accuracy_score /= Validation_times
        print ("Average Train Accuracy :: ", Train_accuracy_score)
        print ("Average Test Accuracy  :: ", Test_accuracy_score)
    
    return trained_model

# Use 'gender'

In [74]:
# 6F

x_feature = ['FN1','FN2','FN1_sonin_index','FN2_sonin_index','FN1_muin_index','FN2_muin_index']
y_feature = ['gender']
    
gender_test = basic_RFClassifier(resampled_gender, x_feature, y_feature, 10)

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.9625527426160337
Test Accuracy  ::  0.8565880872942078


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.9620956399437413
Test Accuracy  ::  0.8596236941420993


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.9618026254102203
Test Accuracy  ::  0.8595689985232183


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.9620018752930145
Test Accuracy  ::  0.8553027402505059


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.9613689639006094
Test Accuracy  ::  0.8563966526281245
Average Train Accuracy ::  0.9620979840600092
Average Test Accuracy  ::  0.8569107914456053


In [75]:
# Read uni-gram classifier
# 6F

x_feature = ['FN1','FN2','FN1_sonin_index','FN2_sonin_index','FN1_muin_index','FN2_muin_index']
    
result = gender_test.predict(resample_taiwandata[x_feature])
prob = gender_test.predict_proba(resample_taiwandata[x_feature])

In [76]:
gender_list = resample_taiwandata['GuessedGender'].tolist()

In [77]:
counter = 0
not_match = []
for x in range(len(result)):
    if gender_list[x] == int(result[x]):
        counter += 1
    else:
        not_match.append(x)
        
print(counter)
print(len(result))
print(counter/len(result))

531
985
0.5390862944162437


In [79]:
# 2F

x_feature = ['FN1','FN2']
y_feature = ['gender']
    
gender_test = basic_RFClassifier(resampled_gender, x_feature, y_feature, 10)

result = gender_test.predict(resample_taiwandata[x_feature])
prob = gender_test.predict_proba(resample_taiwandata[x_feature])

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.9620487576183778
Test Accuracy  ::  0.8374446206858831


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.9610056258790436
Test Accuracy  ::  0.8409451402942625


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.9618495077355836
Test Accuracy  ::  0.8372531860197998


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.9612869198312236
Test Accuracy  ::  0.8378274900180496


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.9611814345991562
Test Accuracy  ::  0.8378001422086091
Average Train Accuracy ::  0.9615564932020628
Average Test Accuracy  ::  0.8380408029316853


In [80]:
counter = 0
not_match = []
for x in range(len(result)):
    if gender_list[x] == int(result[x]):
        counter += 1
    else:
        not_match.append(x)
        
print(counter)
print(len(result))
print(counter/len(result))

295
985
0.29949238578680204


In [81]:
# FN + sonin

x_feature = ['FN1','FN2','FN1_sonin_index','FN2_sonin_index']

y_feature = ['gender']
    
gender_test = basic_RFClassifier(resampled_gender, x_feature, y_feature, 10)

result = gender_test.predict(resample_taiwandata[x_feature])
prob = gender_test.predict_proba(resample_taiwandata[x_feature])

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.9622948898265354
Test Accuracy  ::  0.8431329650494995


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.961966713548992
Test Accuracy  ::  0.848821309413116


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.9618377871542428
Test Accuracy  ::  0.8422851829568452


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.9622597280825129
Test Accuracy  ::  0.844691790187606


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.9616033755274261
Test Accuracy  ::  0.8447738336159274
Average Train Accuracy ::  0.9620335208626349
Average Test Accuracy  ::  0.8447656292730953


In [83]:
counter = 0
not_match = []
for x in range(len(result)):
    if gender_list[x] == int(result[x]):
        counter += 1
    else:
        not_match.append(x)
        
print(counter)
print(len(result))
print(counter/len(result))

295
985
0.29949238578680204


In [84]:
# FN + muin

x_feature = ['FN1','FN2','FN1_muin_index','FN2_muin_index']

y_feature = ['gender']
    
gender_test = basic_RFClassifier(resampled_gender, x_feature, y_feature, 10)

result = gender_test.predict(resample_taiwandata[x_feature])
prob = gender_test.predict_proba(resample_taiwandata[x_feature])

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.9620721987810595
Test Accuracy  ::  0.846168571897391


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.9622128457571496
Test Accuracy  ::  0.8464147021823551


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.9623652133145804
Test Accuracy  ::  0.846524093420117


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.9618612283169246
Test Accuracy  ::  0.8445277033309632


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.9622831692451945
Test Accuracy  ::  0.8479188317015807
Average Train Accuracy ::  0.9620311767463667
Average Test Accuracy  ::  0.8460263632883006


In [85]:
counter = 0
not_match = []
for x in range(len(result)):
    if gender_list[x] == int(result[x]):
        counter += 1
    else:
        not_match.append(x)
        
print(counter)
print(len(result))
print(counter/len(result))

335
985
0.3401015228426396


In [86]:
# sonin + muin

x_feature = ['FN1_sonin_index','FN2_sonin_index','FN1_muin_index','FN2_muin_index']

y_feature = ['gender']
    
gender_test = basic_RFClassifier(resampled_gender, x_feature, y_feature, 10)

result = gender_test.predict(resample_taiwandata[x_feature])
prob = gender_test.predict_proba(resample_taiwandata[x_feature])

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.8660103141115799
Test Accuracy  ::  0.8059946398293497


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.8655297702766057
Test Accuracy  ::  0.8070885522069682


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.8666901078293483
Test Accuracy  ::  0.8052562489744571


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.8656352555086733
Test Accuracy  ::  0.8067056828748017


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.8668073136427566
Test Accuracy  ::  0.8045178581195647
Average Train Accuracy ::  0.8658309892170651
Average Test Accuracy  ::  0.8062845266094186


In [87]:
counter = 0
not_match = []
for x in range(len(result)):
    if gender_list[x] == int(result[x]):
        counter += 1
    else:
        not_match.append(x)
        
print(counter)
print(len(result))
print(counter/len(result))

546
985
0.5543147208121827


In [105]:
# sonin

x_feature = ['FN1_sonin_index','FN2_sonin_index']

y_feature = ['gender']
    
gender_test = basic_RFClassifier(resampled_gender, x_feature, y_feature, 10)

result = gender_test.predict(resample_taiwandata[x_feature])
prob = gender_test.predict_proba(resample_taiwandata[x_feature])

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.6826300984528832
Test Accuracy  ::  0.6806049335448231


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.6833333333333333
Test Accuracy  ::  0.6773778920308483


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.6816104078762306
Test Accuracy  ::  0.6818082371602034


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.682653539615565
Test Accuracy  ::  0.6799759339276924


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.6842358180965776
Test Accuracy  ::  0.6752174150850517
Average Train Accuracy ::  0.6832852789498358
Average Test Accuracy  ::  0.6780287698955314


In [106]:
counter = 0
not_match = []
for x in range(len(result)):
    if gender_list[x] == int(result[x]):
        counter += 1
    else:
        not_match.append(x)
        
print(counter)
print(len(result))
print(counter/len(result))

295
985
0.29949238578680204


In [107]:
# muin

x_feature = ['FN1_muin_index','FN2_muin_index']

y_feature = ['gender']
    
gender_test = basic_RFClassifier(resampled_gender, x_feature, y_feature, 10)

result = gender_test.predict(resample_taiwandata[x_feature])
prob = gender_test.predict_proba(resample_taiwandata[x_feature])

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.7210618846694796
Test Accuracy  ::  0.7173877372422469


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.7221167369901547
Test Accuracy  ::  0.7144888694415578


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.7211908110642288
Test Accuracy  ::  0.7161844336268665


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.7211087669948429
Test Accuracy  ::  0.7167313898156757


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.721390060947023
Test Accuracy  ::  0.7152546081058907
Average Train Accuracy ::  0.7212669948429442
Average Test Accuracy  ::  0.7163485204835093


In [108]:
counter = 0
not_match = []
for x in range(len(result)):
    if gender_list[x] == int(result[x]):
        counter += 1
    else:
        not_match.append(x)
        
print(counter)
print(len(result))
print(counter/len(result))

581
985
0.5898477157360406


# Use 'mgender'

In [102]:
# 6F

x_feature = ['FN1','FN2','FN1_sonin_index','FN2_sonin_index','FN1_muin_index','FN2_muin_index']
y_feature = ['mgender']
    
gender_test = basic_RFClassifier(resampled_gender, x_feature, y_feature, 10)

result = gender_test.predict(resample_taiwandata[x_feature])
prob = gender_test.predict_proba(resample_taiwandata[x_feature])

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.9996601031411158
Test Accuracy  ::  0.9134715309303725


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.9996366619784341
Test Accuracy  ::  0.9117486189356233


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.9996718237224567
Test Accuracy  ::  0.9114751408412186


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.9995897796530708
Test Accuracy  ::  0.9095607941803862


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.99971870604782
Test Accuracy  ::  0.9109555324618498
Average Train Accuracy ::  0.9996390060947024
Average Test Accuracy  ::  0.9107121369578298


In [103]:
counter = 0
not_match = []
for x in range(len(result)):
    if gender_list[x] == int(result[x]):
        counter += 1
    else:
        not_match.append(x)
        
print(counter)
print(len(result))
print(counter/len(result))

482
985
0.48934010152284263


In [104]:
# 2F

x_feature = ['FN1','FN2']
y_feature = ['mgender']
    
gender_test = basic_RFClassifier(resampled_gender, x_feature, y_feature, 10)

result = gender_test.predict(resample_taiwandata[x_feature])
prob = gender_test.predict_proba(resample_taiwandata[x_feature])

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.9987458977965307
Test Accuracy  ::  0.8912924574741563


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.9987693389592124
Test Accuracy  ::  0.8931247607066674


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.9986872948898265
Test Accuracy  ::  0.8912651096647158


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.9988396624472574
Test Accuracy  ::  0.8928239348028223


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.9988162212845757
Test Accuracy  ::  0.8919488049007275
Average Train Accuracy ::  0.9988138771683076
Average Test Accuracy  ::  0.8920746048241537


In [91]:
counter = 0
not_match = []
for x in range(len(result)):
    if gender_list[x] == int(result[x]):
        counter += 1
    else:
        not_match.append(x)
        
print(counter)
print(len(result))
print(counter/len(result))

295
985
0.29949238578680204


In [92]:
# FN + sonin

x_feature = ['FN1','FN2','FN1_sonin_index','FN2_sonin_index']

y_feature = ['mgender']
    
gender_test = basic_RFClassifier(resampled_gender, x_feature, y_feature, 10)

result = gender_test.predict(resample_taiwandata[x_feature])
prob = gender_test.predict_proba(resample_taiwandata[x_feature])

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.9990975152367557
Test Accuracy  ::  0.8979379751681891


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.9993319268635724
Test Accuracy  ::  0.9020127987748181


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.9991092358180966
Test Accuracy  ::  0.8994147568779741


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.9992381622128458
Test Accuracy  ::  0.8959689328884757


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.9993319268635724
Test Accuracy  ::  0.8985122791664387
Average Train Accuracy ::  0.9992651195499296
Average Test Accuracy  ::  0.8985478313187114


In [93]:
counter = 0
not_match = []
for x in range(len(result)):
    if gender_list[x] == int(result[x]):
        counter += 1
    else:
        not_match.append(x)
        
print(counter)
print(len(result))
print(counter/len(result))

295
985
0.29949238578680204


In [94]:
# FN + muin

x_feature = ['FN1','FN2','FN1_muin_index','FN2_muin_index']

y_feature = ['mgender']
    
gender_test = basic_RFClassifier(resampled_gender, x_feature, y_feature, 10)

result = gender_test.predict(resample_taiwandata[x_feature])
prob = gender_test.predict_proba(resample_taiwandata[x_feature])

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.9993084857008908
Test Accuracy  ::  0.8995788437346168


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.9993553680262541
Test Accuracy  ::  0.8999070174479025


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.99943741209564
Test Accuracy  ::  0.8978559317398677


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.99943741209564
Test Accuracy  ::  0.8990865831646885


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.9993084857008908
Test Accuracy  ::  0.8985396269758792
Average Train Accuracy ::  0.999342475386779
Average Test Accuracy  ::  0.9000847782092654


In [95]:
counter = 0
not_match = []
for x in range(len(result)):
    if gender_list[x] == int(result[x]):
        counter += 1
    else:
        not_match.append(x)
        
print(counter)
print(len(result))
print(counter/len(result))

373
985
0.3786802030456853


In [96]:
# sonin + muin

x_feature = ['FN1_sonin_index','FN2_sonin_index','FN1_muin_index','FN2_muin_index']

y_feature = ['mgender']
    
gender_test = basic_RFClassifier(resampled_gender, x_feature, y_feature, 10)

result = gender_test.predict(resample_taiwandata[x_feature])
prob = gender_test.predict_proba(resample_taiwandata[x_feature])

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.8877168307548055
Test Accuracy  ::  0.8314554504184215


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.8878809188935771
Test Accuracy  ::  0.8307170595635289


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.8888185654008439
Test Accuracy  ::  0.831072581086255


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.8862400375058603
Test Accuracy  ::  0.8335065361264563


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.8880684481950305
Test Accuracy  ::  0.8310999288956955
Average Train Accuracy ::  0.8877449601500235
Average Test Accuracy  ::  0.831365202647268


In [97]:
counter = 0
not_match = []
for x in range(len(result)):
    if gender_list[x] == int(result[x]):
        counter += 1
    else:
        not_match.append(x)
        
print(counter)
print(len(result))
print(counter/len(result))

513
985
0.5208121827411167


In [98]:
# sonin

x_feature = ['FN1_sonin_index','FN2_sonin_index']

y_feature = ['mgender']
    
gender_test = basic_RFClassifier(resampled_gender, x_feature, y_feature, 10)

result = gender_test.predict(resample_taiwandata[x_feature])
prob = gender_test.predict_proba(resample_taiwandata[x_feature])

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.6950656352555087
Test Accuracy  ::  0.6887272329486408


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.695182841068917
Test Accuracy  ::  0.6903134058961877


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.6950656352555087
Test Accuracy  ::  0.6900672756112235


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.6944678856071261
Test Accuracy  ::  0.6903681015150687


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.6948195030473512
Test Accuracy  ::  0.6905321883717114
Average Train Accuracy ::  0.6948148148148148
Average Test Accuracy  ::  0.6902641798391949


In [99]:
counter = 0
not_match = []
for x in range(len(result)):
    if gender_list[x] == int(result[x]):
        counter += 1
    else:
        not_match.append(x)
        
print(counter)
print(len(result))
print(counter/len(result))

296
985
0.300507614213198


In [100]:
# muin

x_feature = ['FN1_muin_index','FN2_muin_index']

y_feature = ['mgender']
    
gender_test = basic_RFClassifier(resampled_gender, x_feature, y_feature, 10)

result = gender_test.predict(resample_taiwandata[x_feature])
prob = gender_test.predict_proba(resample_taiwandata[x_feature])

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.7318565400843882
Test Accuracy  ::  0.7233222118908276


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.7303680262541022
Test Accuracy  ::  0.7273423398785758


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.731586966713549
Test Accuracy  ::  0.7240332549362796


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.7308368495077355
Test Accuracy  ::  0.7253459497894219


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.7307313642756681
Test Accuracy  ::  0.7264398621670404
Average Train Accuracy ::  0.7310150023441163
Average Test Accuracy  ::  0.725769840835749


In [101]:
counter = 0
not_match = []
for x in range(len(result)):
    if gender_list[x] == int(result[x]):
        counter += 1
    else:
        not_match.append(x)
        
print(counter)
print(len(result))
print(counter/len(result))

580
985
0.5888324873096447
